# Deep Neural Network for MNIST Classification

The dataset is called MNIST and refers to handwritten digit recognition. 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 


## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [12]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

### Choose the optimizer and the loss function

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [14]:
NUM_EPOCHS = 10

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/10
540/540 - 2s - loss: 0.3292 - accuracy: 0.9009 - val_loss: 0.1958 - val_accuracy: 0.9392 - 2s/epoch - 4ms/step
Epoch 2/10
540/540 - 1s - loss: 0.1315 - accuracy: 0.9606 - val_loss: 0.1055 - val_accuracy: 0.9672 - 1s/epoch - 2ms/step
Epoch 3/10
540/540 - 1s - loss: 0.0929 - accuracy: 0.9713 - val_loss: 0.0810 - val_accuracy: 0.9763 - 1s/epoch - 2ms/step
Epoch 4/10
540/540 - 1s - loss: 0.0753 - accuracy: 0.9769 - val_loss: 0.0764 - val_accuracy: 0.9748 - 1s/epoch - 2ms/step
Epoch 5/10
540/540 - 1s - loss: 0.0626 - accuracy: 0.9808 - val_loss: 0.0653 - val_accuracy: 0.9807 - 1s/epoch - 3ms/step
Epoch 6/10
540/540 - 1s - loss: 0.0522 - accuracy: 0.9833 - val_loss: 0.0597 - val_accuracy: 0.9813 - 1s/epoch - 2ms/step
Epoch 7/10
540/540 - 1s - loss: 0.0485 - accuracy: 0.9845 - val_loss: 0.0634 - val_accuracy: 0.9815 - 1s/epoch - 2ms/step
Epoch 8/10
540/540 - 1s - loss: 0.0403 - accuracy: 0.9877 - val_loss: 0.0535 - val_accuracy: 0.9845 - 1s/epoch - 3ms/step
Epoch 9/10
540/540 - 1s 

## Test the model

In [15]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 98ms/step - loss: 0.0893 - accuracy: 0.9764


In [16]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.64%
